In [5]:
from crud import *
from data import *
import statsmodels.api as sm
import matplotlib.pyplot as plt
from signal_utils import *
from scipy.signal import argrelextrema

In [6]:
mdb = MetadataDB()
tickers = mdb.get_all_tickers()
prices = load_eod_prices()

n-day price windows, see if there is an edge on any securities

In [11]:
df = []

for n in [1, 3, 5, 10, 20, 40, 60]:
    for ticker in tickers:
        if len(prices[ticker]) < 5 * n:
            continue
        t_prices = prices[ticker]["close"]
        windows = [t_prices[i:i+n+1] for i in range(0, len(t_prices), n)]
        n_day_returns = []
        for window in windows:
            n_day_returns.append( (window[-1] / window[0]) - 1)
        
        uu, dd, ud, du = [], [], [], [] # up up, down down, up down, down up
        for i in range(1, len(n_day_returns)):
            if n_day_returns[i-1] > 0:
                if n_day_returns[i] > 0:
                    uu.append(n_day_returns[i])
                else:
                    ud.append(n_day_returns[i])
            else:
                if n_day_returns[i] > 0:
                    du.append(n_day_returns[i])
                else:
                    dd.append(n_day_returns[i])
        total = len(uu) + len(dd) + len(ud) + len(du)

        df.append([ticker, n, total, len(uu)/total, len(dd)/total, len(ud)/total, len(du)/total])

df = pd.DataFrame(df, columns=["ticker", "window", "num_samples", "uu", "dd", "ud", "du"])
df.to_csv("data/exp/dis_matrix.csv", index=False)                



trend, avg drawdown pct, avg up move pct, avg cycle time